In [116]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [117]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv.zip' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

# Data cleaning and normalisation

In [118]:
# we know that very low values are used to signal unavailable data
tX[tX < -900] = np.nan

In [119]:
# split feature 22 by value as planned
tX = np.hstack((np.delete(tX, 22,axis=1), np.stack([tX[:,22] == 0, tX[:,22] == 1, tX[:,22] == 2, tX[:,22] == 3]).T))
print(np.shape(tX))

(250000, 33)


In [120]:
from proj1_helpers import sample_data
seed = 1

tX_mean = np.nanmean(tX,axis=0)

tX_std = np.nanstd(tX,axis=0)
norm_tX = np.subtract(tX, tX_mean, where=np.isfinite(tX_mean))
norm_tX = np.divide(norm_tX, tX_std, where=tX_std>0)
norm_tX[np.isnan(norm_tX)] =0



#norm_tX = np.c_[np.ones((y.shape[0], 1)), norm_tX]
initial_w = np.zeros((norm_tX.shape[1], 1))

print(np.shape(norm_tX))



(250000, 33)


In [121]:
# outliers: we remove any datapoint that is more than 4 standard derivations removed from the mean
# since normalisation was applied, this translate to
norm_tX = np.delete(norm_tX, np.linalg.norm(norm_tX, ord=np.inf, axis=1) > 4, axis=0)


(237360, 33)

# Logistic Regression


In [42]:
from Implementation import logistic_regression_gd

# init parameters
max_iter = 1500
gamma = 0.00001
norm_tX = np.c_[np.ones((y.shape[0], 1)), norm_tX]
initial_w = np.zeros((norm_tX.shape[1], 1))
y[y==-1] = 0

print(np.shape(y))
print(np.shape(norm_tX))
print(np.shape(initial_w))
print(y)
    
weights, loss_lr = logistic_regression_gd(y, norm_tX, initial_w, max_iter, gamma)

#print(weights)

(250000,)
(250000, 34)
(34, 1)
[1. 0. 0. ... 1. 0. 0.]
Current iteration=0, loss=173286.79513997794, gamma=1e-05
Current iteration=100, loss=173272.9512896622, gamma=1e-05
Current iteration=200, loss=173259.22002903727, gamma=1e-05
Current iteration=300, loss=173245.60055404122, gamma=1e-05
Current iteration=400, loss=173232.09206760937, gamma=1e-05
Current iteration=500, loss=173218.69377961237, gamma=1e-05
Current iteration=600, loss=173205.40490678584, gamma=1e-05
Current iteration=700, loss=173192.2246726617, gamma=1e-05
Current iteration=800, loss=173179.15230749932, gamma=1e-05
Current iteration=900, loss=173166.1870482183, gamma=1e-05
Current iteration=1000, loss=173153.32813833142, gamma=1e-05
Current iteration=1100, loss=173140.5748278781, gamma=1e-05
Current iteration=1200, loss=173127.9263733594, gamma=1e-05
Current iteration=1300, loss=173115.38203767297, gamma=1e-05
Current iteration=1400, loss=173102.9410900489, gamma=1e-05
loss=173090.60280598645


# Regularized logistic regression


In [45]:
from Implementation import regu_logistic_regression_gd

# init parameters
max_iter = 1600
gamma = 0.00001
#gamma = 0.00000009
lambda_ = 0.1
norm_tX = np.c_[np.ones((y.shape[0], 1)), norm_tX]


initial_w = np.zeros((norm_tX.shape[1], 1))


weights, loss_rlr = regu_logistic_regression_gd(y, norm_tX, lambda_, initial_w, max_iter, gamma )
#print(weights)

Current iteration=0, loss=173286.79513997794
Current iteration=100, loss=134255.5107390551
Current iteration=200, loss=132938.5845122488
Current iteration=300, loss=132605.87844800614
Current iteration=400, loss=132474.29865345795
Current iteration=500, loss=132402.6074748882
Current iteration=600, loss=132355.89267910525
Current iteration=700, loss=132322.38299321564
Current iteration=800, loss=132296.9784879584
Current iteration=900, loss=132277.07564612041
Current iteration=1000, loss=132261.17952086788
Current iteration=1100, loss=132248.34256149895
Current iteration=1200, loss=132237.91188336047
Current iteration=1300, loss=132229.4078330733
Current iteration=1400, loss=132222.46210773868
Current iteration=1500, loss=132216.78391975214
loss=132211.14764576976


# Least squares and Polynomial regression


In [ ]:
#weights = polynomial_regression(norm_tX)
#weights = polynomial_regression(norm_tX2)

# Ridge regression with a polynome 7 degree



In [122]:
from Implementation import split_data, build_poly, ridge_regression, compute_mse

def ridge_regression_split(x, y), lambda:
    """ridge regression demo."""
   
    # define parameter
    degree = 7
    seed = 29
    ratio = 0.8
    lambda_ = 0.1
     
    # split data
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)
     
    # form tx
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)
    
    # ridge regression with different lambda
    rmse_tr = []
    rmse_te = []
    
    # ridge regression
    
    weight, loss_rmse = ridge_regression(y_tr, tx_tr, lambda_)
    rmse_tr.append(np.sqrt(2 * compute_mse(y_tr, tx_tr, weight)))
    rmse_te.append(np.sqrt(2 * compute_mse(y_te, tx_te, weight)))

    print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
           p=ratio, d=degree, l=lambda_, tr=rmse_tr[0], te=rmse_te[0]))
    
    return weight, loss_rmse

In [123]:
weights = ridge_regression_split(norm_tX, y)
#print(weights)
#print(np.shape(weights))

proportion=0.8, degree=7, lambda=0.100, Training RMSE=0.777, Testing RMSE=0.893
[-4.05284362e-03  1.10836875e-01 -1.64612064e-01  1.84564099e-02
  3.83275656e-02  1.40727829e-02  1.56753986e-02 -2.42532676e-02
  5.13592051e-02 -2.83875210e-02  4.42799676e-02 -8.31254786e-02
  2.23265309e-02  3.19802415e-02  1.17954743e-01 -1.45474120e-03
 -3.52183970e-04  2.57893504e-02  1.34224232e-03  1.06619805e-03
 -4.03257755e-03  1.24685716e-03  1.24447543e-02  2.44537568e-02
 -1.84593729e-03 -7.97776361e-04  1.55485982e-02 -1.77448347e-04
 -7.77658991e-04  1.94949894e-02 -3.76654956e-03  1.47909317e-03
  1.99811822e-03  1.26394275e-03 -1.27467973e-01  1.87252879e-02
 -8.90480247e-02  3.57948982e-02  2.30983421e-02  1.83105095e-02
  4.64451312e-03  4.55265768e-03  4.61305795e-03 -2.39421523e-02
  5.74505332e-02  9.12288283e-03  5.63885272e-03 -4.14755690e-02
 -7.78831042e-03 -3.90538050e-03 -8.20514121e-03 -2.11755877e-02
 -2.54387975e-03  5.47019659e-02 -9.61990983e-04 -3.67408823e-02
 -2.794583

# Split data Log Res



In [97]:
from Implementation import train_test_split_demo_lr
seed = 6
degrees = [7]
split_ratio = 0.8
#split_ratios = [0.7, 0.8 , 0.9]
weights = train_test_split_demo_lr(norm_tX, y, split_ratio, seed)

(200000,)
(200000, 34)
(34, 1)
Current iteration=0, loss=138629.4361119834, gamma=9e-08
Current iteration=100, loss=138629.33514666208, gamma=9e-08
Current iteration=200, loss=138629.23418874817, gamma=9e-08
Current iteration=300, loss=138629.13323824687, gamma=9e-08
Current iteration=400, loss=138629.0322951576, gamma=9e-08
Current iteration=500, loss=138628.93135948002, gamma=9e-08
Current iteration=600, loss=138628.8304312135, gamma=9e-08
Current iteration=700, loss=138628.7295103576, gamma=9e-08


KeyboardInterrupt: 

## Generate predictions and save ouput in csv format for submission:

In [124]:
DATA_TEST_PATH = 'data/test.csv.zip' # TODO: download train data and supply path here 
y, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [125]:
# we know that very low values are used to signal unavailable data
tX_test[tX_test < -900] = np.nan

In [126]:
# split feature 22 by value as planned
tX_test = np.hstack((np.delete(tX_test, 22,axis=1), np.stack([tX_test[:,22] == 0, tX_test[:,22] == 1, tX_test[:,22] == 2, tX_test[:,22] == 3]).T))
print(np.shape(tX_test))

(568238, 33)


In [127]:

norm_tX_test = np.subtract(tX_test, tX_mean, where=np.isfinite(tX_mean))
norm_tX_test = np.divide(norm_tX_test, tX_std, where=tX_std>0)
norm_tX_test[np.isnan(norm_tX_test)] =0
#norm_tX_test = np.c_[np.ones((y.shape[0], 1)), norm_tX_test]
norm_tX_test = build_poly(norm_tX_test, 7)

print(np.shape(norm_tX_test))
print(np.shape(weights))
print(np.shape(y))



(568238, 232)
(232,)
(568238,)


In [129]:
OUTPUT_PATH = 'data/sample-submission' # TODO: fill in desired name of output file for submission
#print(np.shape(y))
#print(np.shape(norm_tX_test))
#print(np.shape(weights))
#print(weights)
y_pred = predict_labels(weights, norm_tX_test)

create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[-1.18598128 -0.57506826 -0.18612134 ...  0.02961414  0.08455748
 -0.69189037]
